In [ ]:
import shutil
from glob import glob
import nrrd
import os
import PIL
from PIL import Image
from torch.utils.data import Dataset
import torch
import pandas as pd
import numpy as np
import matplotlib as plot
from matplotlib import pylab as plt
from preprocess.utils import *

In [ ]:
import SimpleITK as sitk

"""
    以下x,y,z方向最大半径分割
    结合临床数据保存为numpy数组，[0]为切片数据,[1]为临床数据
"""
#读取excel文件
OUTSIZE=[32,32,32]
data_path=r'F:\Download\1'
csv_path=os.path.join(data_path,'results_20220822.xlsx')
data=pd.read_excel(csv_path,usecols='B:I')
exfiles=['09652616']
print(data)

In [ ]:
#存在背景信息

error_file=[]
#image_path存放nrrd文件
image_path=os.path.join(data_path,'image')
for i in range(len(data)):
    filename=str(data.iloc[i,0])[:-5]
    cdata=np.array(data.iloc[i,:])
    if filename in exfiles:
        continue
    filepath=os.path.join(data_path,filename+'.nrrd')
    mask_file_path=os.path.join(data_path,filename+'_mask.nrrd')
    type=int(data.iloc[i,1])

    image_file=os.path.join(image_path,filename)
    origin_data=sitk.ReadImage(filepath)
    mask_data=sitk.ReadImage(mask_file_path)
    #若mask文件和nrrd文件不匹配，跳过
    if mask_data.GetSize()!=origin_data.GetSize():
        error_file.append(filename)
        continue
    #获取结节占据的x,y,z轴切片序号
    #注意sitk.GetArrayFromImage将图片又[x,y,z]变为[z,x,y]
    nonIndex=getnonzeros(sitk.GetArrayFromImage(mask_data))
    #设置窗口并缩放到[0,255]
    #origin_data=setWindow(origin_data)
    #连通域分割
    mask_data=sitk.ConnectedComponent(mask_data)
    label_shape_filter = sitk.LabelShapeStatisticsImageFilter()
    label_shape_filter.Execute(mask_data)
    #分割最大连通域
    area_max_label=0
    area_max=0
    for i in range(1, label_shape_filter.GetNumberOfLabels()+1):
        area=label_shape_filter.GetNumberOfPixels(i)
        if area>area_max:
            area_max_label = i
            area_max = area
    centor=origin_data.TransformPhysicalPointToIndex(label_shape_filter.GetCentroid(area_max_label))
    x,y,z,w,h,d=label_shape_filter.GetBoundingBox(area_max_label)
    r=int(max(w,h,d)*1.2)
    start_x,start_y,start_z=int(x-(r-w)/2),int(y-(r-h)/2),int(z-(r-d)/2)
    ROI_data=sitk.RegionOfInterest(origin_data,index=[start_x,start_y,start_z],size=[r,r,r])

    ROI_data=resample2Size(ROI_data,OUTSIZE)
    ndata=sitk.GetArrayFromImage(ROI_data)
    target_path=os.path.join(image_path,f'{type}')
    if not os.path.exists(target_path):
        os.makedirs(target_path)
    np.savez(os.path.join(target_path,filename+'.npz'),image=ndata,cdata=np.zeros(shape=(5,)))
    pic_path=os.path.join(image_path,'pic')
    if not os.path.exists(pic_path):
        os.makedirs(pic_path)
    ndata=zscore(ndata)*255
    img=Image.fromarray(ndata[int(OUTSIZE[2]/2),:,:])
    img.convert('L').save(os.path.join(pic_path,f'{filename}_z.png'))
    print(filename,':complete!')
print('错误文件：',error_file)

In [5]:
#不存在背景信息

error_file=[]
#image_path存放nrrd文件
image_path=os.path.join(data_path,'image2')
for i in range(len(data)):
    filename=str(data.iloc[i,0])[:-5]
    cdata=np.array(data.iloc[i,:])
    if filename in exfiles:
        continue
    filepath=os.path.join(data_path,filename+'.nrrd')
    mask_file_path=os.path.join(data_path,filename+'_mask.nrrd')
    type=int(data.iloc[i,1])

    image_file=os.path.join(image_path,filename)
    origin_data=sitk.ReadImage(filepath)
    mask_data=sitk.ReadImage(mask_file_path)
    #若mask文件和nrrd文件不匹配，跳过
    if mask_data.GetSize()!=origin_data.GetSize():
        error_file.append(filename)
        continue
    #获取结节占据的x,y,z轴切片序号
    #注意sitk.GetArrayFromImage将图片又[x,y,z]变为[z,x,y]
    nonIndex=getnonzeros(sitk.GetArrayFromImage(mask_data))
    #设置窗口并缩放到[0,255]
    origin_data=setWindow(origin_data)
    #连通域分割
    mask_data=sitk.ConnectedComponent(mask_data)
    label_shape_filter = sitk.LabelShapeStatisticsImageFilter()
    label_shape_filter.Execute(mask_data)
    #分割最大连通域
    area_max_label=0
    area_max=0
    for j in range(1, label_shape_filter.GetNumberOfLabels()+1):
        area=label_shape_filter.GetNumberOfPixels(j)
        if area>area_max:
            area_max_label = j
            area_max = area
    centor=origin_data.TransformPhysicalPointToIndex(label_shape_filter.GetCentroid(area_max_label))
    x,y,z,w,h,d=label_shape_filter.GetBoundingBox(area_max_label)
    r=max(w,h,d)
    #清楚背景信息
    Ndata=sitk.GetArrayFromImage(origin_data)
    Ndata_mask=sitk.GetArrayFromImage(mask_data)
    node_Ndata=Ndata*Ndata_mask
    node_Ndata[node_Ndata==0]=0
    node_data=sitk.GetImageFromArray(node_Ndata)
    start_x,start_y,start_z=int(x-(r-w)/2),int(y-(r-h)/2),int(z-(r-d)/2)
    ROI_data=sitk.RegionOfInterest(node_data,index=[start_x,start_y,start_z],size=[r,r,r])

    ROI_data=resample2Size(ROI_data,OUTSIZE)
    ndata=sitk.GetArrayFromImage(ROI_data)
    target_path=os.path.join(image_path,f'{type}')
    if not os.path.exists(target_path):
        os.makedirs(target_path)
    np.savez(os.path.join(target_path,filename+'.npz'),image=ndata,cdata=np.zeros(shape=(5,)))
    pic_path=os.path.join(image_path,'pic')
    if not os.path.exists(pic_path):
        os.makedirs(pic_path)
    ndata=zscore(ndata)*255
    img=Image.fromarray(ndata[int(OUTSIZE[2]/2),:,:])
    img.convert('L').save(os.path.join(pic_path,f'{filename}_z.png'))
    print(f'{i}:',filename,':complete!')
print('错误文件：',error_file)


0: 00625858 :complete!
1: 01034481 :complete!
2: 02501737 :complete!
3: 03022694 :complete!
4: 03223027 :complete!
5: 03571004 :complete!
6: 04183261 :complete!
7: 04247827 :complete!
8: 04595852 :complete!
9: 04693158 :complete!
10: 04776614 :complete!
11: 04873120 :complete!
12: 05118583 :complete!
13: 05256788 :complete!
14: 05376764 :complete!
15: 05412252 :complete!
16: 05431443 :complete!
17: 05545138 :complete!
18: 05587338 :complete!
19: 05688162 :complete!
20: 05738378 :complete!
21: 05807742 :complete!
22: 05849467 :complete!
23: 05871891 :complete!
24: 05930651 :complete!
25: 05968948 :complete!
26: 06025976 :complete!
27: 06135499 :complete!
28: 06137339 :complete!
29: 06260590 :complete!
30: 07056211 :complete!
31: 07136826 :complete!
32: 07156597 :complete!
33: 07156637 :complete!
34: 07157615 :complete!
35: 07238217 :complete!
36: 07280060 :complete!
37: 07289679 :complete!
38: 07459320 :complete!
39: 07592297 :complete!
40: 07723928 :complete!
41: 07870634 :complete!
42